In [2]:
import os
os.environ['NUMBA_ENABLE_CUDASIM'] = "1"

# playing with porting to CUDA

In [31]:
import numba.cuda as cuda
import numba as nb
import numpy as np
import math
from math import asinh, sqrt, atan
from time import time

eps = 1e-18

# newell f
def f(p):
  #print(type(p))
  x, y, z = abs(p[0]), abs(p[1]), abs(p[2])
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
def g(p):
  x, y, z = p[0], p[1], abs(p[2])
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0


from math import pi
from time import time

_file = open("orig.txt", "w")
# demag tensor setup
def set_n_demag_orig(c, permute, func, n_demag, dx):
  it = np.nditer(n_demag[:,:,:,c], flags=['multi_index'], op_flags=['writeonly'])
  #_file.write("-"*64 + "\n")
  while not it.finished:
    value = 0.0
    for i in np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6):
      idx = list(map(lambda k: (it.multi_index[k] + n[k] - 1) % (2*n[k] - 1) - n[k] + 1, range(3)))
      vec = list(map(lambda j: (idx[j] + i[j] - i[j+3]) * dx[j], permute))
      #_file.write(f"{i}\t\t{vec}\t\t\t{-1**sum(i)}\t\t\t{-1 ** sum(i) * func((vec[0], vec[1], vec[2]))}\n")
      value += (-1)**sum(i) * func((vec[0], vec[1], vec[2]))
      #_file.write(f"v = {v}, value = {value}\n")
      #_file.write(f"{i} : {(-1)**sum(i)}\n")
    it[0] = - value / (4 * pi * np.prod(dx))
    #_file.write(f"{4 * pi * np.prod(dx)}\n")
    #_file.write(f"arr[{it.multi_index}]={it[0]}\n")
    #_file.write("-"*64 + "\n")
    it.iternext()


def calculate_demag_tensor_orig(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])

    for i, t in enumerate(((f,0,1,2),(g,0,1,2),(g,0,2,1),(f,1,2,0),(g,1,2,0),(f,2,0,1))):
        set_n_demag_orig(i, t[1:], t[0], n_demag=n_demag, dx=dx)
    return n_demag

In [32]:
t = time()
n     = (100, 25, 1)
dx    = (5e-9, 5e-9, 3e-9)

f = f_orig
g = g_orig
tensor_original = calculate_demag_tensor_orig(n ,dx)
print(f"number of cells : {np.prod(n)}")
print(f"calculated demag tensor in : {time() - t}")
_file.close()

Calculating the demagnetization tensor
number of cells : 2500
calculated demag tensor in : 99.62391185760498


In [29]:
np.save("demag_tensor.npy",tensor_original)

In [33]:
true = np.load("demag_tensor_orig.npy")

In [51]:
# newell f
#@cuda.jit

eps=1e-18
PI = pi

_file = open("cuda_out.txt", "w")

@cuda.jit
def f_cuda(p0, p1, p2):
  #print(type(p))
  x, y, z = abs(p0), abs(p1), abs(p2)
  return + y / 2.0 * (z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         + z / 2.0 * (y**2 - x**2) * asinh(z / (sqrt(x**2 + y**2) + eps)) \
         - x*y*z * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))       \
         + 1.0 / 6.0 * (2*x**2 - y**2 - z**2) * sqrt(x**2 + y**2 + z**2)

# newell g
@cuda.jit
def g_cuda(p0, p1, p2):
  x, y, z = p0, p1, abs(p2)
  return + x*y*z * asinh(z / (sqrt(x**2 + y**2) + eps))                         \
         + y / 6.0 * (3.0 * z**2 - y**2) * asinh(x / (sqrt(y**2 + z**2) + eps)) \
         + x / 6.0 * (3.0 * z**2 - x**2) * asinh(y / (sqrt(x**2 + z**2) + eps)) \
         - z**3 / 6.0 * atan(x*y / (z * sqrt(x**2 + y**2 + z**2) + eps))        \
         - z * y**2 / 2.0 * atan(x*z / (y * sqrt(x**2 + y**2 + z**2) + eps))    \
         - z * x**2 / 2.0 * atan(y*z / (x * sqrt(x**2 + y**2 + z**2) + eps))    \
         - x*y * sqrt(x**2 + y**2 + z**2) / 3.0
@cuda.jit
def demag_calc_gpu(array, idxes, n, permute, dx, _func, idx_table):
    x_idx, y_idx, z_idx = cuda.grid(3)
    if x_idx < array.shape[0] and y_idx < array.shape[1] and z_idx < array.shape[2]:
        #print(x_idx, y_idx, z_idx)
        idx = idx_table[x_idx, y_idx, z_idx,:]
        value = 0
        i = 0
        #_file.write("-"*64 + "\n")
        #_file.write(f"x,y,z = {(x_idx, y_idx,z_idx)}")
        while i < 64:
            idx[0] = (x_idx + n[0] - 1) % (2*n[0] - 1) - n[0] + 1
            idx[1] = (y_idx + n[1] - 1) % (2*n[1] - 1) - n[1] + 1
            idx[2] = (z_idx + n[2] - 1) % (2*n[2] - 1) - n[2] + 1


            x = (idx[permute[0]] + idxes[i][permute[0]] - idxes[i][permute[0]+3]) * dx[permute[0]]
            y = (idx[permute[1]] + idxes[i][permute[1]] - idxes[i][permute[1]+3]) * dx[permute[1]]
            z = (idx[permute[2]] + idxes[i][permute[2]] - idxes[i][permute[2]+3]) * dx[permute[2]]

            sign = (-1)**(idxes[i][0] + idxes[i][1] + idxes[i][2] + idxes[i][3] + idxes[i][4] + idxes[i][5])
            v = 0
            if _func[0] == 0:
                #_file.write(f"{list(idxes[i])}\t\t{(x,y,z)}\t\t\t{-1**sum(idxes[i])}\t\t\t{sign * f_cuda(x,y,z)}\n")
                #print(type(f_cuda(x,y,z)))
                value += sign * f_cuda(x,y,z)
            else:
                #_file.write(f"{list(idxes[i])}\t\t{(x,y,z)}\t\t\t{-1**sum(idxes[i])}\t\t\t{sign * g_cuda(x,y,z)}\n")
                #print(type(g_cuda(x,y,z)))
                value += sign * g_cuda(x,y,z)
            #_file.write(f"{list(idxes[i])} {sign}\n")
            #_file.write(f"v = {v}, val = {value}\n")

            i += 1
        div = 4 * PI * dx[0] * dx[1] * dx[2]
        array[x_idx,y_idx,z_idx] = - value / div
        #_file.write(f"{div}\n")
        #_file.write(f"arr[{x_idx},{y_idx},{z_idx}] = {array[x_idx, y_idx, z_idx]}\n")
        #_file.write("-"*64 + "\n")


# demag tensor setup
def set_n_demag_cuda(permute, func, dx):
    threadsperblock = (32, 8, 1)
    an_array = np.zeros(( 2*n[0]-1, 2*n[1] - 1, 2 * n[2]-1), dtype=np.float64)
    arr_cuda = cuda.to_device(an_array)
    n_cuda = cuda.to_device(np.array(n, dtype=int))
    permute_cuda = cuda.to_device(np.array(permute, dtype=int))
    dx_cuda = cuda.to_device(np.array(dx, dtype=np.float64))
    idxes = np.rollaxis(np.indices((2,)*6), 0, 7).reshape(64, 6)
    idxes_cuda = cuda.to_device(idxes)
    idx_cuda = np.zeros(an_array.shape + (3, ))
    idx_cuda = cuda.to_device(idx_cuda)

    if func == 'f':
        #print(0)
        _func = np.array([0])
    else:
        #print(1)
        _func = np.array([1])

    _func = cuda.to_device(_func)
    blockspergrid_x = math.ceil(an_array.shape[0] / threadsperblock[0])
    blockspergrid_y = math.ceil(an_array.shape[1] / threadsperblock[1])
    blockspergrid_z = math.ceil(an_array.shape[2] / threadsperblock[2])
    blockspergrid = (blockspergrid_x, blockspergrid_y, blockspergrid_z)

    demag_calc_gpu[blockspergrid, threadsperblock](arr_cuda, idxes_cuda, n_cuda, permute_cuda, dx_cuda, _func, idx_cuda)
    return arr_cuda

def identity(p):
    return 0

def calculate_demag_tensor_cuda(n, dx):
    print("Calculating the demagnetization tensor")
    n_demag = np.zeros([2*i-1 for i in n] + [6])
    res = []

    for i, t in enumerate((('f',0,1,2),('g',0,1,2),('g',0,2,1),('f',1,2,0),('g',1,2,0),('f',2,0,1))):
        r = set_n_demag_cuda(func=t[0], permute=t[1:], dx=dx).copy_to_host()
        res.append(np.expand_dims(r, 3))
    return np.concatenate(res, axis=3)

n = (100, 25, 1)
dx    = (5e-9, 5e-9, 3e-9)

t = time()
f = f_cuda
g = g_cuda

print(f"how many cells : {np.prod(n)}")
cuda_calc = calculate_demag_tensor_cuda(n ,dx)
np.save("demag_tensor_gpu.npy",cuda_calc)
print(cuda_calc.shape)
_file.close()
print(f"Took {time() - t} s")
np.mean(np.abs(cuda_calc -true))

how many cells : 2500
Calculating the demagnetization tensor
(199, 49, 1, 6)
Took 1.8777172565460205 s


5.036521025943494e-12